In [1]:
import torch
import torchvision 
from torchvision import datasets
import torchvision.transforms as t
import torch.utils.data as data
import torch.nn.functional as f
import numpy as np

#check that gpu is used for NN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
trans_total = t.Compose([t.Grayscale(),
                        t.ToTensor(),
                        t.Normalize((0.5), (0.5)),
                        t.Pad(6)])

FashionData = datasets.MNIST(root = "./", train = True, download=False, transform=trans_total)

In [3]:
FashionData[0][0].size()

torch.Size([1, 40, 40])

In [4]:
FashionData_size = len(FashionData)
train_size = FashionData_size * 0.7
valid_size = FashionData_size - train_size
print(FashionData_size, train_size, valid_size)
train_set, valid_set = data.random_split(FashionData, [int(train_size), int(valid_size)], generator=torch.Generator().manual_seed(31))


60000 42000.0 18000.0


In [5]:
BATCH_SIZE = 100
loader = data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
batch = next(iter(loader))
images_batch, labels_batch = batch
print(images_batch.shape, labels_batch.shape)

torch.Size([100, 1, 40, 40]) torch.Size([100])


In [9]:
class NetWork(torch.nn.Module):
    def __init__(self):
        super(NetWork, self).__init__()
        # torch.nn has classes for every common deep learning layer:
        # linear layers, convolutional layers, transformers, etc.
        
        
        
        #input Size : 40x40x1
        #output Size : 19x19x16
        #output width = (40 - 4)/2 + 1 =  19
        self.conv1 = torch.nn.Conv2d(1,8, kernel_size = 4, stride = 2)
        
        #maxpooling 
        #input Size : 19x19x16
        #output Size : 9x9x16        
        #output width = (19 - 3)/2 + 1 =  9
        self.pool1 = torch.nn.MaxPool2d(kernel_size = 3, stride = 2)
        
        #input Size : 9x9x16
        #output Size : 7x7x32
        #output width = (9 - 3)/1 + 1 =  7
        self.conv2 = torch.nn.Conv2d(8, 12, kernel_size = 3, stride = 1)
        
        #maxpooling 
        #input Size : 7x7x32
        #output Size : 6x6x32        
        #output width = (7 - 2)/1 + 1 =  6
        self.pool2 = torch.nn.MaxPool2d(kernel_size = 2, stride = 1)
        
        
        self.linear1 = torch.nn.Linear(432, 128, bias=True) # defines a single linear layer
        self.linear2 = torch.nn.Linear(128, 10, bias=True)

    def forward(self, x):
        # the Module.forward function is where you define how your network transforms input features to output
        # predictions
        
        x = f.relu(self.conv1(x))
        #print(x.shape)
        x = self.pool1(x)
        #print(x.shape)
        x = f.relu(self.conv2(x))
        #print(x.shape)
        x = self.pool2(x)                
        #print(x.shape)
        x = torch.flatten(x,1 )
        #print(x.shape)
        x = f.relu(self.linear1(x))
        x = self.linear2(x)
        
        #print(x.shape)                    
        return x

In [20]:
net = NetWork()
LR = 1e-3

pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(pytorch_total_params)

optim = torch.optim.SGD(net.parameters(), lr=LR)
loss_func = torch.nn.CrossEntropyLoss()

57726


In [21]:
EPOCHS = 200

val_losses = []
for epoch in range(EPOCHS):
    print(epoch)
    for batch_idx, batch in enumerate(loader):
        image_batch, lable_batch = batch
        lablehat_batch = net(image_batch)

        loss = loss_func(lablehat_batch, lable_batch)
        optim.zero_grad()
        loss.backward()
        optim.step()

        # lr_scheduler.step()
    
    yhat_val = net(image_batch)
    val_loss = loss_func(lablehat_batch, lable_batch)
    val_losses.append(val_loss.detach().item())
    print(val_loss)
    if(val_loss < .01):
        break;


import os
torch.save(net.state_dict(), 'model.ckpt')

0
tensor(2.2991, grad_fn=<NllLossBackward>)
1
tensor(2.2921, grad_fn=<NllLossBackward>)
2
tensor(2.2886, grad_fn=<NllLossBackward>)
3
tensor(2.2732, grad_fn=<NllLossBackward>)
4
tensor(2.2542, grad_fn=<NllLossBackward>)
5
tensor(2.2455, grad_fn=<NllLossBackward>)
6
tensor(2.2063, grad_fn=<NllLossBackward>)
7
tensor(2.1731, grad_fn=<NllLossBackward>)
8
tensor(2.1144, grad_fn=<NllLossBackward>)
9
tensor(1.9853, grad_fn=<NllLossBackward>)
10
tensor(1.7717, grad_fn=<NllLossBackward>)
11
tensor(1.4824, grad_fn=<NllLossBackward>)
12
tensor(1.1987, grad_fn=<NllLossBackward>)
13
tensor(0.9156, grad_fn=<NllLossBackward>)
14
tensor(0.9991, grad_fn=<NllLossBackward>)
15
tensor(0.6454, grad_fn=<NllLossBackward>)
16
tensor(0.7699, grad_fn=<NllLossBackward>)
17
tensor(0.6615, grad_fn=<NllLossBackward>)
18
tensor(0.5670, grad_fn=<NllLossBackward>)
19
tensor(0.5657, grad_fn=<NllLossBackward>)
20
tensor(0.4454, grad_fn=<NllLossBackward>)
21
tensor(0.4928, grad_fn=<NllLossBackward>)
22
tensor(0.4046, gr

tensor(0.0325, grad_fn=<NllLossBackward>)
182
tensor(0.0678, grad_fn=<NllLossBackward>)
183
tensor(0.1663, grad_fn=<NllLossBackward>)
184
tensor(0.0334, grad_fn=<NllLossBackward>)
185
tensor(0.1395, grad_fn=<NllLossBackward>)
186
tensor(0.0242, grad_fn=<NllLossBackward>)
187
tensor(0.1081, grad_fn=<NllLossBackward>)
188
tensor(0.0608, grad_fn=<NllLossBackward>)
189
tensor(0.0935, grad_fn=<NllLossBackward>)
190
tensor(0.0528, grad_fn=<NllLossBackward>)
191
tensor(0.0707, grad_fn=<NllLossBackward>)
192
tensor(0.0225, grad_fn=<NllLossBackward>)
193
tensor(0.0950, grad_fn=<NllLossBackward>)
194
tensor(0.0709, grad_fn=<NllLossBackward>)
195
tensor(0.0625, grad_fn=<NllLossBackward>)
196
tensor(0.0546, grad_fn=<NllLossBackward>)
197
tensor(0.0327, grad_fn=<NllLossBackward>)
198
tensor(0.0886, grad_fn=<NllLossBackward>)
199
tensor(0.0759, grad_fn=<NllLossBackward>)


In [22]:
correct = 0;

valid_loader = data.DataLoader(valid_set, batch_size=int(valid_size), shuffle=False)
for data_val_idx, data_val in enumerate(valid_loader):
        image_data_val, lable_data_val = data_val
        lablehat_data_val = net(image_data_val)
        prediction = lablehat_data_val.argmax(dim=1)
        
        for i in range (int(valid_size)) :
            if(prediction[i] == lable_data_val[i]):
                correct = correct + 1
print("acc_val = ", correct/valid_size)

acc_val =  0.9754444444444444


In [23]:
correct = 0;
FashionData_test = datasets.MNIST(root = "./", train = False, download=False, transform=trans_total)
test_size = len(FashionData_test)
test_loader = data.DataLoader(FashionData_test, batch_size=test_size, shuffle=False)
for data_val_idx, data_val in enumerate(test_loader):
        image_data_val, lable_data_val = data_val
        lablehat_data_val = net(image_data_val)
        prediction = lablehat_data_val.argmax(dim=1)

        for i in range (test_size):
            if(prediction[i] == lable_data_val[i]):
                correct = correct + 1
print("acc_val = ", correct/test_size)

acc_val =  0.9769
